In [3]:
!{'python3 -m pip install dataflows==0.2.2'}

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import getpass
CKAN_API_KEY = getpass.getpass("CKAN_API_KEY")

CKAN_API_KEY ····································


In [21]:
CKAN_URL = 'https://www.odata.org.il'

In [23]:
import shutil
import requests
import dataflows as DF

shutil.rmtree('.checkpoints/all_resources', ignore_errors=True)

def get_datasets():
    limit = 1000
    offset = 0
    while True:
        last_offset = offset
        for package in requests.get(CKAN_URL+'/api/3/action/package_search?include_private=true&rows={}&start={}'.format(limit, offset), headers={'X-CKAN-API-Key': CKAN_API_KEY}).json()['result']['results']:
            for resource in package.get('resources', []):
                yield {
                    'mimetype': resource['mimetype'],
                    'name': resource['name'],
                    'url': resource['url'],
                    'format': resource['format'],
                    'state': resource['state'],
                    'id': resource['id'],
                    'size': resource['size'],
                    'package_id': package['id'],
                    'package_private': package['private'],
                    'package_state': package['state'],
                    'package_type': package['type'],
                    'package_name': package['name'],
                    'package_title': package['title']
                }
            offset += 1
        if last_offset == offset:
            break
        else:
            print(offset)

DF.Flow(
    get_datasets(),
    DF.checkpoint('all_resources'),
    DF.printer(tablefmt='html', num_rows=1)
).process()

saving checkpoint to: .checkpoints/all_resources


1000
2000
3000
4000
4174


#,mimetype (string),name (string),url (string),format (string),state (string),id (string),size (integer),package_id (string),package_private (boolean),package_state (string),package_type (string),package_name (string),package_title (string)
1,text/csv,existing_entities,https://www.odata.org.il/dataset/9dfa6da6-5826-4f53-802b-b588323f44f3/resource/b93234e9-a0b9-4356-87 ...,CSV,active,b93234e9-a0b9-4356-8707-22ea9ac83d40,18514560,9dfa6da6-5826-4f53-802b-b588323f44f3,True,active,dataset,all_entities_list,רשימת רשויות / קבוצות / יישויות
2,text/csv,טיפות חלב חלוקה לאזורים בעיר-דצמ' 2020,https://www.odata.org.il/dataset/d7d7644e-19d5-4e48-bb16-47ed6703cdbe/resource/76dd6c0a-67ef-494b-ae ...,CSV,active,76dd6c0a-67ef-494b-aed0-a49d2b003bca,1543,d7d7644e-19d5-4e48-bb16-47ed6703cdbe,False,active,dataset,jerusalem_dgpsync_tipot-halav-areas,עיריית ירושלים - טיפות חלב חלוקה לאזורים בעיר
...,,,,,,,,,,,,,
12438,None,מרכז ישיבות בני עקיבא בישראל - ביקורת עומק.pdf,https://www.odata.org.il/dataset/e42f7a84-f666-42d6-8b69-803317bfe30c/resource/77a81b3c-0de2-4e5a-83 ...,PDF,active,77a81b3c-0de2-4e5a-83ba-bc001e1a481c,None,e42f7a84-f666-42d6-8b69-803317bfe30c,False,active,dataset,bikoret-bneiakiva,ביקורת עומק בעמותת מרכז ישיבות בני עקיבא בישראל


checkpoint saved: all_resources


(<datapackage.package.Package at 0x7f48eb99c908>, {})

In [48]:
from collections import defaultdict

include_formats = ['PDF', 'JPEG', 'GIF', 'PNG', 'TIFF', 'BMP']
exclude_formats = ['CSV', 'XSL', 'WEB', 'XLSX', 'ICS', 'ZIP', 'XLS', 'DOC', 'DOCX', 'GEOJSON', 'WORD', 'GIS, SHAPEFILE', 'XML', 'RAR', 'PPTX', 
                   'MESSAGE/RFC822', 'TXT', 'API', 'APPLICATION/VND.MS-EXCEL.SHEET.BINARY.MACROENABLED.12', 'KMZ', 'JSON', 'APPLICATION/X-MS-WMZ',
                  'PPT', 'APPLICATION/X-7Z-COMPRESSED', '']

def filter_resources_for_ocr(rows):
    stats = defaultdict(int)
    for row in rows:
        if row['package_state'] != 'active':
            stats['package state not active'] += 1
            continue
        elif row['package_type'] != 'dataset':
            stats['package type not dataset'] += 1
            continue
        elif not row['url'] or len(row['url']) < 10:
            stats['invalid package url'] += 1
            continue
        elif row['state'] != 'active':
            stats['resource state not active']
            continue
        format = row['format']
        format = format.upper() if format else ''
        if format in include_formats:
            stats['included format'] += 1
            yield row
        elif format in exclude_formats:
            stats['excluded format'] += 1
        else:
            stats['unknown format'] += 1
            print('unknown format: {}'.format(format))
    print(dict(stats))
    if stats['unknown format'] > 0:
        raise Exception("Encountered unknown format!")

DF.Flow(
    DF.checkpoint('all_resources'),
    filter_resources_for_ocr,
    DF.printer(tablefmt='html', num_rows=1)
).process()

using checkpoint data from .checkpoints/all_resources


{'excluded format': 3973, 'invalid package url': 4, 'included format': 8461}


#,mimetype (string),name (string),url (string),format (string),state (string),id (string),size (integer),package_id (string),package_private (boolean),package_state (string),package_type (string),package_name (string),package_title (string)
1,image/jpeg,2B3BC97A1F654CE8A1F66FA6BC394491[509628].jpg,https://www.odata.org.il/dataset/619f4b50-6804-4c2e-80dc-7740bff7e45a/resource/9c59682d-5b63-412c-a9 ...,JPEG,active,9c59682d-5b63-412c-a9a2-6fe2fa6dfa53,10878,619f4b50-6804-4c2e-80dc-7740bff7e45a,True,active,dataset,ad72c989-688f-4c09-9556-0258f02a411c,משטרת ישראל - התקשרויות לשנת 2020 (רבעון ראשון)
2,image/gif,9C5F3CBAE0684C32AF1B9A8B7E6520B8[509627].gif,https://www.odata.org.il/dataset/619f4b50-6804-4c2e-80dc-7740bff7e45a/resource/c34dfa82-cc41-4b6a-93 ...,GIF,active,c34dfa82-cc41-4b6a-9350-bd1d1e02ef18,8158,619f4b50-6804-4c2e-80dc-7740bff7e45a,True,active,dataset,ad72c989-688f-4c09-9556-0258f02a411c,משטרת ישראל - התקשרויות לשנת 2020 (רבעון ראשון)
...,,,,,,,,,,,,,
8461,None,מרכז ישיבות בני עקיבא בישראל - ביקורת עומק.pdf,https://www.odata.org.il/dataset/e42f7a84-f666-42d6-8b69-803317bfe30c/resource/77a81b3c-0de2-4e5a-83 ...,PDF,active,77a81b3c-0de2-4e5a-83ba-bc001e1a481c,None,e42f7a84-f666-42d6-8b69-803317bfe30c,False,active,dataset,bikoret-bneiakiva,ביקורת עומק בעמותת מרכז ישיבות בני עקיבא בישראל


(<datapackage.package.Package at 0x7f48eb840f28>, {})

In [56]:
import os

def http_stream_download(filename, **requests_kwargs):
    with requests.get(stream=True, **requests_kwargs) as res:
        res.raise_for_status()
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'wb') as f:
            for chunk in res.iter_content(chunk_size=8192):
                if chunk:  # filter out keep-alive new chunks
                    f.write(chunk)

def download_resources(rows):
    failed_resource_ids = []
    stats = defaultdict(int)
    for row in rows:
        package_id = row['package_id']
        resource_id = row['id']
        if resource_id in failed_resource_ids:
            continue
        format = row['format'].lower()
        filename = os.path.join('data', 'export_for_ocr_processing', package_id[0], package_id[1], package_id, '{}.{}'.format(resource_id, format))
        if not os.path.exists(filename):
            try:
                http_stream_download(filename, url=row['url'], headers={'Authorization': CKAN_API_KEY})
            except Exception:
                print('failed to download resource_id: {}'.format(resource_id))
                failed_resource_ids.append(resource_id)
                shutil.rmtree(filename, ignore_errors=True)
                continue
        stats['bytes'] += os.path.getsize(filename)
        print(stats['bytes'])
        yield row
    print(dict(stats))
    print('failed_resource_ids:')
    print(failed_resource_ids)

DF.Flow(
    DF.checkpoint('all_resources'),
    filter_resources_for_ocr,
    download_resources,
    DF.printer(tablefmt='html', num_rows=1)
).process()

using checkpoint data from .checkpoints/all_resources


10878
19036
2389428
2837024
2847902
2856060
3400644
3456893
3463213
3519454
3590005
3778838
4084199
4089006
4090560
4092511
4093395
4093997
4094753
4101073
4109345
4546432
4547502
4549929
4551148
4552453
4697483
19874461
19880781
20027501
20033821
20181088
20182158
20184585
20185804
20187109
20334261
20417183
20756452
20762772
21453175
21791039
22150753
22524895
22863734
22866055
22869425
22873756
23215384
23287966
23387988
23394308
24563422
24566286
24572606
24962948
24967017
25136260
25632693
25639013
26341409
26347729
26350954
26351739
26352719
26762106
26775398
26781718
30203405
30226838
43660490
45522742
45524000
45525143
45526462
45527811
45528963
45530813
45561149
45570773
45965051
45971371
46130256
46135832
46141408
46144358
50427872
50428942
50429222
50433612
50435100
50436475
50437694
50438999
50445319
50499868
50578468
50580791
50790082
50796402
50799374
50805900
50806085
50817950
50831000
50837498
51417679
51423999
51425841
52351584
52361208
52362434
52363562
52364740
52365

#,mimetype (string),name (string),url (string),format (string),state (string),id (string),size (integer),package_id (string),package_private (boolean),package_state (string),package_type (string),package_name (string),package_title (string)
1,image/jpeg,2B3BC97A1F654CE8A1F66FA6BC394491[509628].jpg,https://www.odata.org.il/dataset/619f4b50-6804-4c2e-80dc-7740bff7e45a/resource/9c59682d-5b63-412c-a9 ...,JPEG,active,9c59682d-5b63-412c-a9a2-6fe2fa6dfa53,10878,619f4b50-6804-4c2e-80dc-7740bff7e45a,True,active,dataset,ad72c989-688f-4c09-9556-0258f02a411c,משטרת ישראל - התקשרויות לשנת 2020 (רבעון ראשון)
2,image/gif,9C5F3CBAE0684C32AF1B9A8B7E6520B8[509627].gif,https://www.odata.org.il/dataset/619f4b50-6804-4c2e-80dc-7740bff7e45a/resource/c34dfa82-cc41-4b6a-93 ...,GIF,active,c34dfa82-cc41-4b6a-9350-bd1d1e02ef18,8158,619f4b50-6804-4c2e-80dc-7740bff7e45a,True,active,dataset,ad72c989-688f-4c09-9556-0258f02a411c,משטרת ישראל - התקשרויות לשנת 2020 (רבעון ראשון)
...,,,,,,,,,,,,,
8349,None,מרכז ישיבות בני עקיבא בישראל - ביקורת עומק.pdf,https://www.odata.org.il/dataset/e42f7a84-f666-42d6-8b69-803317bfe30c/resource/77a81b3c-0de2-4e5a-83 ...,PDF,active,77a81b3c-0de2-4e5a-83ba-bc001e1a481c,None,e42f7a84-f666-42d6-8b69-803317bfe30c,False,active,dataset,bikoret-bneiakiva,ביקורת עומק בעמותת מרכז ישיבות בני עקיבא בישראל


(<datapackage.package.Package at 0x7f48eb21e780>, {})